This is my "sandbox" to play around with the BEA api. My plan is to use this for my regional cycle project. Below I explain how to use the BEA api (stuff has changed since I last messed around with it)

In [46]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt

In [53]:
BEA_ID = "insert here"

After getting your key. [Here is some basic documentation](https://www.bea.gov/API/bea_web_service_api_user_guide.htm). Like a lot APIs, basic idea is to specify the URL in the proper way and it will return some output in a specified format. We can then convert the format (its .json) into a dataframe:



In [4]:
API_URL = "https://bea.gov/api/data?&"
API_URL = API_URL + "UserID=" + BEA_ID + "&method=GETDATASETLIST&ResultFormat=JSON&"
    
r = requests.get(API_URL)

In [5]:
type(r)


requests.models.Response

In [6]:
type(r.json())

dict

In [7]:
print(r.json().keys())

print(r.json()['BEAAPI'].keys())

print(r.json()['BEAAPI']["Results"].keys())

dict_keys(['BEAAPI'])
dict_keys(['Request', 'Results'])
dict_keys(['Dataset'])


This then leads to the insight that the `.json()` is a bunch of dictionaries within the dictionaries and that by working through it we can find a "root dictionary" that can be converted into a usable DataFrame

In [8]:
df = pd.DataFrame(r.json()["BEAAPI"]["Results"]['Dataset'])

In [9]:
df

,DatasetDescription,DatasetName
0,The RegionalData dataset is obsolete. Please u...,RegionalData
1,Standard NIPA tables,NIPA
2,Standard NI underlying detail tables,NIUnderlyingDetail
3,Multinational Enterprises,MNE
4,Standard Fixed Assets tables,FixedAssets
5,International Transactions Accounts,ITA
6,International Investment Position,IIP
7,GDP by Industry,GDPbyIndustry
8,Regional Income data sets,RegionalIncome
9,Regional Product data sets,RegionalProduct


Then this gives us the different datasets that are available through the BEA api. Now below, I'm going to grab personal income at the county level. The documentation for grabing this is [here](https://www.bea.gov/API/bea_web_service_api_user_guide.htm)

---

Now what I want to do is to (for all years possible), grab the [personal per capital income](https://www.bea.gov/regional/#defs) and the population measure at the country level. There are several steps here. First, I construct a string of all the years I want. Then I pass the appropriate URL to the BEA. This took some time to figure this all out.

In [25]:
years = range(1969,2018)
years = list(years)

years = "".join(str(years))

years = years[1:-1]

In [26]:
years

'1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017'

This then grabs the per capita personal income data:

In [27]:
my_key = "https://bea.gov/api/data?&UserID=" + BEA_ID + "&method=GetData&"

data_set = "datasetname=RegionalIncome&"

table_and_line_income = "TableName=CA1&LineCode=3&"

table_and_line_population = "TableName=CA1&LineCode=2&"

year = "Year=" + years + "&"

location = "GeoFips=COUNTY&"

form = "ResultFormat=json"

In [28]:
API_URL = my_key + data_set + table_and_line_income + year + location + form

r = requests.get(API_URL)

df_income = pd.DataFrame(r.json()["BEAAPI"]["Results"]["Data"])

In [29]:
df_income.head(45)

,CL_UNIT,Code,DataValue,GeoFips,GeoName,NoteRef,TimePeriod,UNIT_MULT
0,dollars,CA1-3,15459,00000,United States,NaN,1986,0
1,dollars,CA1-3,4196,00000,United States,NaN,1970,0
2,dollars,CA1-3,10153,00000,United States,NaN,1980,0
3,dollars,CA1-3,19591,00000,United States,NaN,1990,0
4,dollars,CA1-3,30602,00000,United States,NaN,2000,0
5,dollars,CA1-3,41082,00000,United States,NaN,2008,0
6,dollars,CA1-3,6312,00000,United States,NaN,1975,0
7,dollars,CA1-3,12652,00000,United States,NaN,1983,0
8,dollars,CA1-3,8338,00000,United States,NaN,1978,0
9,dollars,CA1-3,17426,00000,United States,NaN,1988,0


In [30]:
df_income.drop(['CL_UNIT', 'Code',"NoteRef", "UNIT_MULT"], axis=1, inplace = True)

#df["DataValue"].column = "IncomePC"

df_income.rename(columns={"DataValue":"IncomePC"}, inplace=True)

df_income.head()

,IncomePC,GeoFips,GeoName,TimePeriod
0,15459,00000,United States,1986
1,4196,00000,United States,1970
2,10153,00000,United States,1980
3,19591,00000,United States,1990
4,30602,00000,United States,2000


In [31]:
API_URL = my_key + data_set + table_and_line_population + year + location + form

r = requests.get(API_URL)

population = pd.DataFrame(r.json()["BEAAPI"]["Results"]["Data"])

population.drop(['CL_UNIT', 'Code',"NoteRef", "UNIT_MULT", "GeoName"], axis=1, inplace = True)

#df["DataValue"].column = "IncomePC"

population.rename(columns={"DataValue":"Population"}, inplace=True)

population.head()

,Population,GeoFips,TimePeriod
0,203798722,00000,1970
1,217553859,00000,1976
2,213333635,00000,1974
3,227224719,00000,1980
4,311663358,00000,2011


In [32]:
combo = pd.merge(population, df_income,   # left df, right df
                 how='inner',      # Try the different options, inner, outer, left, right...what happens.
                 on=['GeoFips',"TimePeriod"],       # link with cntry
                 indicator=True)  # Tells us what happend

In [33]:
combo["TimePeriod"] = pd.to_datetime(combo["TimePeriod"], infer_datetime_format = True)

#combo.sort_values(by = "TimePeriod", inplace = True)

In [34]:
combo.set_index(["GeoFips","TimePeriod"],inplace = True)

In [35]:
combo.sort_index(level="GeoFips", inplace = True)

In [40]:
combo.head(20)

Population IncomePC        GeoName _merge
GeoFips TimePeriod                                          
00000   1969-01-01  201298000     3930  United States   both
        1970-01-01  203798722     4196  United States   both
        1971-01-01  206817509     4468  United States   both
        1972-01-01  209274882     4853  United States   both
        1973-01-01  211349205     5352  United States   both
        1974-01-01  213333635     5824  United States   both
        1975-01-01  215456585     6312  United States   both
        1976-01-01  217553859     6856  United States   both
        1977-01-01  219760875     7494  United States   both
        1978-01-01  222098244     8338  United States   both
        1979-01-01  224568579     9212  United States   both
        1980-01-01  227224719    10153  United States   both
        1981-01-01  229465744    11262  United States   both
        1982-01-01  231664432    11947  United States   both
        1983-01-01  233792014    12652  United States   both
        1984-01-01  235824907    13860  United States   both
        1985-01-01  237923734    14719  United States   both
        1986-01-01  240132831    15459  United States   both
        1987-01-01  242288936    16265  United States   both
        1988-01-01  244499004    17426  United States   both

In [41]:
combo.Population.replace("(NA)", np.nan, inplace = True)
combo.Population = combo.Population.astype(float)

In [43]:
combo.IncomePC.replace("(NA)", np.nan, inplace = True)
combo.IncomePC = combo.IncomePC.astype(float)

In [52]:
combo.loc[["36061","02020"]]

Population  IncomePC                     GeoName _merge
GeoFips TimePeriod                                                         
02020   1969-01-01    123265.0    6232.0  Anchorage Municipality, AK   both
        1970-01-01    127569.0    6823.0  Anchorage Municipality, AK   both
        1971-01-01    134619.0    7102.0  Anchorage Municipality, AK   both
        1972-01-01    143203.0    7348.0  Anchorage Municipality, AK   both
        1973-01-01    147314.0    7853.0  Anchorage Municipality, AK   both
        1974-01-01    152386.0    9320.0  Anchorage Municipality, AK   both
        1975-01-01    165035.0   11439.0  Anchorage Municipality, AK   both
        1976-01-01    174496.0   12853.0  Anchorage Municipality, AK   both
        1977-01-01    177003.0   14071.0  Anchorage Municipality, AK   both
        1978-01-01    179642.0   13998.0  Anchorage Municipality, AK   both
        1979-01-01    178825.0   14572.0  Anchorage Municipality, AK   both
        1980-01-01    175808.0   16648.0  Anchorage Municipality, AK   both
        1981-01-01    180969.0   18758.0  Anchorage Municipality, AK   both
        1982-01-01    195216.0   21514.0  Anchorage Municipality, AK   both
        1983-01-01    211028.0   21806.0  Anchorage Municipality, AK   both
        1984-01-01    220254.0   22634.0  Anchorage Municipality, AK   both
        1985-01-01    226848.0   23386.0  Anchorage Municipality, AK   both
        1986-01-01    229965.0   23135.0  Anchorage Municipality, AK   both
        1987-01-01    225170.0   22406.0  Anchorage Municipality, AK   both
        1988-01-01    224371.0   22733.0  Anchorage Municipality, AK   both
        1989-01-01    225374.0   24694.0  Anchorage Municipality, AK   both
        1990-01-01    227614.0   25781.0  Anchorage Municipality, AK   both
        1991-01-01    235137.0   26113.0  Anchorage Municipality, AK   both
        1992-01-01    245623.0   26880.0  Anchorage Municipality, AK   both
        1993-01-01    250163.0   27842.0  Anchorage Municipality, AK   both
        1994-01-01    252278.0   28556.0  Anchorage Municipality, AK   both
        1995-01-01    251981.0   28924.0  Anchorage Municipality, AK   both
        1996-01-01    250710.0   29854.0  Anchorage Municipality, AK   both
        1997-01-01    252700.0   31612.0  Anchorage Municipality, AK   both
        1998-01-01    257232.0   32746.0  Anchorage Municipality, AK   both
...                        ...       ...                         ...    ...
36061   1987-01-01   1481531.0   34824.0                New York, NY   both
        1988-01-01   1484183.0   41302.0                New York, NY   both
        1989-01-01   1486046.0   44394.0                New York, NY   both
        1990-01-01   1487073.0   50225.0                New York, NY   both
        1991-01-01   1483644.0   46524.0                New York, NY   both
        1992-01-01   1485877.0   51437.0                New York, NY   both
        1993-01-01   1493582.0   53301.0                New York, NY   both
        1994-01-01   1503909.0   54790.0                New York, NY   both
        1995-01-01   1514184.0   59488.0                New York, NY   both
        1996-01-01   1521415.0   64770.0                New York, NY   both
        1997-01-01   1527384.0   69626.0                New York, NY   both
        1998-01-01   1531067.0   75552.0                New York, NY   both
        1999-01-01   1535624.0   81258.0                New York, NY   both
        2000-01-01   1540547.0   88640.0                New York, NY   both
        2001-01-01   1555729.0   93696.0                New York, NY   both
        2002-01-01   1555382.0   89316.0                New York, NY   both
        2003-01-01   1562154.0   86665.0                New York, NY   both
        2004-01-01   1569947.0   89427.0                New York, NY   both
        2005-01-01   1573573.0   97093.0                New York, NY   both
        2006-01-01   1578171.0  110071.0                New York, NY

In [288]:
df.DataValue.replace("(NA)", np.nan, inplace = True)

In [289]:
df.DataValue = df.DataValue.astype(float)

In [290]:
df.dtypes

CL_UNIT        object
Code           object
DataValue     float64
GeoFips        object
GeoName        object
NoteRef        object
TimePeriod     object
UNIT_MULT      object
dtype: object

In [291]:
df["TimePeriod"] = pd.to_datetime(df["TimePeriod"], infer_datetime_format = True)

In [292]:
df.sort_values(by = "TimePeriod", inplace = True)


In [294]:
df.head()

,CL_UNIT,Code,DataValue,GeoFips,GeoName,NoteRef,TimePeriod,UNIT_MULT
0,dollars,CA1-3,3930.0,00000,United States,NaN,1969-01-01,0
3535,dollars,CA1-3,2800.0,31123,"Morrill, NE",NaN,1969-01-01,0
3536,dollars,CA1-3,3093.0,31125,"Nance, NE",NaN,1969-01-01,0
3539,dollars,CA1-3,3727.0,31127,"Nemaha, NE",NaN,1969-01-01,0
3540,dollars,CA1-3,3106.0,31129,"Nuckolls, NE",NaN,1969-01-01,0


In [346]:
grp = df.groupby("GeoFips")

In [310]:
def log_diff_income(df):
    df["growth"] = np.log(df.DataValue).diff() / 46
    
    return df

In [388]:
transform_dict = {"DataValue": ["first","last", lambda x: np.diff(np.log(x))/46]}

new_df = grp.agg(transform_dict)

In [394]:
new_df["DataValue"]["last"].corr(new_df["DataValue"]["<lambda>"])

0.414791642511866

In [367]:
test =  new_df.diff(axis = 1).toseries

AttributeError: 'DataFrame' object has no attribute 'toseries'

In [385]:
np.diff?

In [309]:
grp = df.groupby("GeoFips")
grp.get_group("44007").TimePeriod.diff()

Object `TimePeriod.diff` not found.


In [ ]:
grp.get_group("44007").TimePeriod.diff

In [ ]:
grp.get_group("44007").sort_value

In [ ]:
grp.get_group("44007").sort_values

In [277]:
new_df.sort_values?

In [332]:
new_df.growth.corr(new_df.loc["1969-01-01"].DataValue)

nan

In [325]:
new_df.set_index("TimePeriod", inplace = True)

In [376]:
df.diff?